In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'2.2.0-rc4'

# Part 1 - Data Preprocessing

In [2]:
# Importing the dataset
%%time
dataset = pd.read_csv('drive/My Drive/ML - AZ Course/Data/Churn_Modelling.csv')
X = dataset.drop(["RowNumber", "CustomerId", "Surname", "Exited"], axis=1)
y = dataset["Exited"]
print(X)
print(y)

      CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0             619    France  Female  ...          1               1        101348.88
1             608     Spain  Female  ...          0               1        112542.58
2             502    France  Female  ...          1               0        113931.57
3             699    France  Female  ...          0               0         93826.63
4             850     Spain  Female  ...          1               1         79084.10
...           ...       ...     ...  ...        ...             ...              ...
9995          771    France    Male  ...          1               0         96270.64
9996          516    France    Male  ...          1               1        101699.77
9997          709    France  Female  ...          0               1         42085.58
9998          772   Germany    Male  ...          1               0         92888.52
9999          792    France  Female  ...          1              

## Encoding categorical data

In [3]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X["Gender"] = le.fit_transform(X["Gender"])
print(X)

      CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0             619    France       0  ...          1               1        101348.88
1             608     Spain       0  ...          0               1        112542.58
2             502    France       0  ...          1               0        113931.57
3             699    France       0  ...          0               0         93826.63
4             850     Spain       0  ...          1               1         79084.10
...           ...       ...     ...  ...        ...             ...              ...
9995          771    France       1  ...          1               0         96270.64
9996          516    France       1  ...          1               1        101699.77
9997          709    France       0  ...          0               1         42085.58
9998          772   Germany       1  ...          1               0         92888.52
9999          792    France       0  ...          1              

In [4]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
print(X)

[[ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167  0.97024255
   0.02188649]
 [-1.00280393 -0.57873591  1.74273971 ... -1.54776799  0.97024255
   0.21653375]
 [ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167 -1.03067011
   0.2406869 ]
 ...
 [ 0.99720391 -0.57873591 -0.57380915 ... -1.54776799  0.97024255
  -1.00864308]
 [-1.00280393  1.72790383 -0.57380915 ...  0.64609167 -1.03067011
  -0.12523071]
 [ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167 -1.03067011
  -1.07636976]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Part 2 - Building the ANN

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu, sigmoid
# Initializing the ANN
ann = Sequential([Dense(units=8, activation=relu), #First input layer (hidden layer)
                  Dense(units=16, activation=relu), #Second input layer (hidden layer)
                  Dense(units=1, activation=sigmoid)] #Output layer
                 )

# Part 3 - Training the ANN

###Early Stopping Callback
Helps prevent overfitting by stopping a model when a certain evaluation metric stops improving. Keep finding patterns until the quality of those patterns starts to go down.

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
# Create early stopping
early_stopping = EarlyStopping(monitor="accuracy",
                               patience=3) # stops after 3 rounds of no improvements

In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
# Compiling the ANN
ann.compile(optimizer = Adam(), 
            loss = BinaryCrossentropy(), 
            metrics = ['accuracy'])

# Training the ANN on the Training set
ann.fit(X_train, y_train, batch_size = 32, epochs = 100, callbacks=[early_stopping])

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5440 - accuracy: 0.7481
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4506 - accuracy: 0.8031
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4280 - accuracy: 0.8142
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4132 - accuracy: 0.8255
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3977 - accuracy: 0.8317
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3855 - accuracy: 0.8394
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3761 - accuracy: 0.8444
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3680 - accuracy: 0.8470
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3611 - accuracy: 0.8511
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3559 - accura

# Part 4 - Making the predictions and evaluating the model

In [20]:
# Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [21]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1499   96]
 [ 189  216]]


Relu activation:<br>
[[1512   83]<br>
 [ 205  200]]<br>
 Swish activation:<br>
 [[1503   92]<br>
 [ 193  212]]<br>
 Selu activation:<br>
 [[1518   77]<br>
 [ 203  202]]<br>
 Tanh activation:<br>
 [[1518   77]<br>
 [ 194  211]]

In [22]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}" )
print(f"F1 Score: {f1_score(y_test, y_pred)}")

Accuracy: 0.8575
Recall: 0.5333333333333333
F1 Score: 0.6025104602510459
